<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/gan/c1_w4_controllable_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Controllable Generation

### Goals

Explore GAN contrallability approaches using gradients from a classifier. By training a classifier to recognize a relevant feature, you can use it to change the generator's input (z-vectors) to make it generate images with more or less of that feature.

### Packages and Visualization

In [ ]:
import torch
import matplotlib.pyplot as plt

from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import CelebA
from torch.utils.data import DataLoader

torch.manual_seed(0)

torch.__version__

'1.7.0+cu101'

In [ ]:
def show_tensor_images(image_tensor, num_images=16, size=(3, 64, 64), nrow=3):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

### Generator and Noise

In [ ]:
class Generator(nn.Module):
    """
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_chan: the number of channels in the images, fitted for the dataset
                 used, a scalar (CelebA is rgb, so 3 is our default)
        hidden_dim: the inner dimension, a scalar
    """

    def __init__(self, z_dim=10, im_chan=3, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(z_dim, hidden_dim * 8),
            self.make_gen_block(hidden_dim * 8, hidden_dim * 4),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2),
            self.make_gen_block(hidden_dim * 2, hidden_dim),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, 
                                final_layer=True),
        )

    def make_gen_block(
        self,
        input_channels,
        output_channels,
        kernel_size=3,
        stride=2,
        final_layer=False,
    ):
        """
        Function to return a sequence of operations corresponding to a generator
        block of DCGAN; a transposed convolution, a batchnorm (except in the 
        final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation 
                            has
            output_channels: how many channels the output feature representation 
                             should have
            kernel_size: the size of each convolutional filter, equivalent to 
                         (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false 
                         otherwise (affects activation and batchnorm)
        """
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(
                    input_channels, output_channels, kernel_size, stride
                ),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(
                    input_channels, output_channels, kernel_size, stride
                ),
                nn.Tanh(),
            )

    def forward(self, noise):
        """
        Function for completing a forward pass of the generator: Given a noise 
        tensor, returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        """
        x = noise.view(len(noise), self.z_dim, 1, 1)
        return self.gen(x)


def get_noise(n_samples, z_dim, device="cpu"):
    """
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
    creates a tensor of that shape filled with random numbers from the normal 
    distribution.
    Parameters:
        n_samples: the number of samples in the batch, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    """
    return torch.randn(n_samples, z_dim, device=device)


### Classifier

In [ ]:
class Classifier(nn.Module):
    """
    Classifier Class
    Values:
        im_chan = the number of channels tin the images, fitted for the dataset
                  used, a scalar
        n_classes: the total number of classes in the dataset, an intger scalar
        hidden_dim: the inner dimension, a scalar
    """
    def __init__(self, im_chan=3, n_classes=2, hidden_dim=64):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            self.make_classifier_block(im_chan, hidden_dim),
            self.make_classifier_block(hidden_dim, hidden_dim * 2),
            self.make_classifier_block(hidden_dim * 2, hidden_dim * 4, stride=3),
            self.make_classifier_block(
                hidden_dim * 4, 
                n_classes, 
                final_layer=True)
        )

    def make_classifier_block(self, input_channels, output_channels, 
                              kernel_size=4, final_layer=False):
         make_classifier_block(self, input_channels, output_channels, kernel_size=4, stride=2, final_layer=False):
        """
        Function to return a sequence of operations corresponding to a 
        classifier block; a convolution, a batchnorm (except in the final 
        layer), and an activation (except in the final layer).
        Parameters:
            input_channels: how many channels the input feature representation 
                            has
            output_channels: how many channels the output feature representation 
                             should have
            kernel_size: the size of each convolutional filter, equivalent to 
                         (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false 
                         otherwise 
        """
        if final_layer:
            return 

